<a href="https://colab.research.google.com/github/dablro12/data_analyses/blob/main/Part2_Avatar2_Final_project_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# (구글 코랩에서 실습할 경우) 아래 코드를 실행하여 한글 폰트를 설치합니다. 
# 설치가 완료되면, 커널을 재시작합니다. (상단 메뉴에서 [런타임] - [런타임 다시 시작] 을 선택)

# (로컬 PC에서 작업할 경우) 이 코드를 실행하지 않습니다!!!

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 27 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-nanum all 20180306-3 [9,599 kB]
Fetched 9,599 kB in 2s (6,137 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

In [2]:
import pandas as pd
import numpy as np

import os, re
from tqdm import tqdm

# 경고문구 미표시
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 지정
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

In [3]:
# 다운로드 받을 폴더를 준비
DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok=True)

In [4]:
# 트레이닝 데이터 셋 다운로드 
import torchtext 
torchtext.utils.download_from_url(url='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', 
                                  path=os.path.join(DATA_DIR, 'review.txt'))

14.6MB [00:00, 73.4MB/s]


'/content/data/review.txt'

In [5]:
# txt 파일을 판다스 데이터프레임으로 읽어오기
data = pd.read_csv('./data/review.txt', sep='\t')

# 데이터프레임의 크기
print(data.shape)

# 첫 5행을 출력
data.head()

(150000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


# 2. KoNLPy 한글 형태소 분석

In [6]:
# 코랩에 konlpy 패키지를 설치합니다.
! pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 38.0 MB/s eta 0:00:00


In [7]:
# 긍정 리뷰를 하나 선택
sample_text = data['document'].iloc[149978]
print(type(sample_text))
print(sample_text)
# 트위터 형태소 분석기(Okt)를 활용
from konlpy.utils import pprint
from konlpy.tag import Okt
okt = Okt()
print(okt)

<class 'str'>
그리 만족스럽진못했어도 7점은 나와야되는것같아 10점줌. 주인공들연기도 훌륭했고 내용도 이정도면 괜찮았다. 해피엔딩으로끝났으면 그저 진부한영화가 되버릴뻔. 그래도 엔딩이 섭섭한건 없지않아있었음. 킬러들의도시도그렇고 콜린파렐을 너무 축축하게 끝내버리는듯


# 3. 텍스트 전처리

In [8]:
# 누락 데이터를 제거
review_data = data['document'].dropna().values

# 학습 속도를 고려하여, 1000개의 샘플을 선택하여 추출
review_data = review_data[:1000]

# 배열의 크기
print(review_data.shape)

# 첫 번째 데이터
print(review_data[0])

(1000,)
아 더빙.. 진짜 짜증나네요 목소리


In [9]:
# 세 글자 이상의 명사를 사용 (두 글자 이하의 단어는 제거)
cleaned_review_data = []

for review in tqdm(review_data):
    tokens = okt.nouns(review)
    cleaned_tokens = []
    for word in tokens:
        if len(word) >= 3:
            cleaned_tokens.append(word)
        else:
            pass
    cleaned_review = " ".join(cleaned_tokens)
    cleaned_review_data.append(cleaned_review)

print(len(cleaned_review_data))
print(cleaned_review_data[0])

100%|██████████| 1000/1000 [00:26<00:00, 37.57it/s]

1000
목소리


# 4. TF-IDF 벡터로 표현

In [10]:
# 사이킷런 패키지 활용 
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 변환기 객체를 생성
tfid = TfidfVectorizer()

# TF-IDF 변환기에 데이터를 입력하여 변환
review_tfid = tfid.fit_transform(cleaned_review_data)

# 배열의 크기
print(review_tfid.shape)

# 첫 번째 데이터
print(review_tfid[0])

(1000, 597)
  (0, 152)	1.0


In [11]:
# 단어 사전 확인 (딕셔너리 형태)
vocab = tfid.vocabulary_

# 단어 사전의 크기
print(len(vocab))

# 단어 사전 출력 (앞에서 5개의 단어만 출력)
print({ k:v for i, (k, v) in enumerate(vocab.items()) if i < 5 })

597
{'목소리': 152, '포스터': 546, '교도소': 31, '이야기': 400, '스파이더맨': 260}


In [12]:
# 단어들의 사전 인덱스를 이용하여 원래 단어를 검색하는 매핑 딕셔너리
index_to_word = { v:k for k, v in vocab.items() } 

# 앞에서 5개의 단어를 출력
print({  k:v for i, (k, v) in enumerate(index_to_word.items()) if i < 5 })

{152: '목소리', 546: '포스터', 31: '교도소', 400: '이야기', 260: '스파이더맨'}


In [13]:
# 첫 번째 리뷰를 구성하는 단어들의 사전 인덱스를 이용하여 원래 단어를 복원 (순서 복원 X)
original_text = " ".join([index_to_word[word_idx] for word_idx in review_tfid[0].indices])
original_text

'목소리'

# Topic Modeling

## 1. LDA

In [14]:
# 사이킷런 패키지 활용
from sklearn.decomposition import LatentDirichletAllocation

# LDA 모델링 객체를 생성 (토픽 갯수를 2로 설정: 긍정/부정)
lda = LatentDirichletAllocation(2)  

# TF-IDF 벡터를 입력하여 모델 학습 
lda.fit(review_tfid)

LatentDirichletAllocation(n_components=2)

In [15]:
# 토픽 모델링 결과를 담고 있는 배열의 형태 : (2개의 토픽(긍정/부정), 597개의 단어)
print(lda.components_.shape)

(2, 597)


In [17]:
# 2157개의 단어 중에서, 토픽 별로 가장 중요도가 높은 단어를 10개씩 출력

for idx, topic in enumerate(lda.components_):
    print(f"토픽 유형 {idx+1}:", [(index_to_word[i], topic[i].round(3)) for i in topic.argsort()[:-11:-1]])

토픽 유형 1: [('스토리', 18.209), ('주인공', 13.799), ('이야기', 11.006), ('마지막', 9.582), ('포스터', 6.376), ('스릴러', 5.131), ('시청률', 5.069), ('공포영화', 4.486), ('로맨스', 4.467), ('최고다', 3.478)]
토픽 유형 2: [('드라마', 19.501), ('쓰레기', 15.717), ('캐스팅', 6.081), ('연기력', 5.239), ('시리즈', 4.942), ('비디오', 4.623), ('긴장감', 4.473), ('어린이', 4.245), ('이영화', 4.232), ('코미디', 4.124)]


## 2. pyLDAvis 시각화

In [18]:
# pyLDAvis 설치
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=dc65e6d42cc876ffb350f14e7dad651c73b6c5654fa7c5ef8a8f784d3a2193fd
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=bbd194da96435b9c0ad97bc4f9382c868844eda1819e5a59a178b553df4860b5
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyLDAvis sklearn


In [ ]:
# LDA 토픽 모델링 결과를 시각화
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
visualization = pyLDAvis.sklearn.prepare(lda, review_tfid, tfid)
pyLDAvis.display(visualization)

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping


# Sentimental Analysis (감성 분석)

## 1. 머신러닝 알고리즘 활용해 긍정/부정 분류

In [21]:
labels = data['label'].iloc[:1000].values
print(labels.shape)
print(labels[:5])

(1000,)
[0 1 0 0 1]


In [22]:
# 사이킷런 패키지 활용
from sklearn.linear_model import LogisticRegression

# 로지스틱 분류 모델링 객체를 생성 
lr = LogisticRegression()

# TF-IDF 벡터를 입력하여 모델 학습 
lr.fit(review_tfid, labels)

LogisticRegression()

In [23]:
# 첫 번째 샘플을 이용하여 모델 예측
pred = lr.predict(review_tfid[0])
print(pred)

[0]


## 2. Part 1에서 수집한 리뷰를 긍정, 부정으로 감성 분류 예측

In [33]:
# 판다스 데이터프레임로 읽어오기
test = pd.read_csv('./data/naver_review_Avatar2.csv')

len(test['review_content'])

10

In [37]:
# 두 번째 리뷰를 선택
test_sample = test['review_content'][1]
test_sample

'2014년에 아바타2가 개봉하다니ㅋ\n\xa0\n아바타1,, 정말 재밌게 봤었는데\n\xa0\n물론 지금 이글을 쓰고있는 시간은 2011년 5월이지만\n\xa0\n다시 생각해도 아바타 정말 걸작중의 걸작이다!!\xa0\n\xa0\n2편은 판도라바다?를 배경으로 만든다는데...\n\xa0\n아무튼 재밌겠다.보러 가야지ㅋㅋ\n\xa0\n현재 2011년에는 3d 4d가 유행인데\n\xa0\n2014년엔 5D 나오려나?ㅋㅋㅋ\n\xa0\n3년뒤니깐 나올수도 있겠지..\n\xa0\n지금으로부터 3년전 2008년만 해도\n\xa0\n영화를 3D로 본다는건 상상도못했었는데ㅋㅋ\n\xa0\n암튼 과학기술의 발달은 정말 빠른것같다ㅡ\n\xa0\n이제 미래에 대한 몇가지 예측들을 해보겠다..\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n2014년\n\xa0\n예측 1.\xa0 (영화)\n\xa0\ni ) 아바타 2\n\xa0\n대부분 1편에서 흥행성공한 영화들은\n\xa0\n후속작에서 망하던데..\n\xa0\n아바타도 과연 그럴까??\n\xa0\n내 생각엔 아바타2도 흥행 성공 할것같다.\n\xa0\n왜냐하면 스케일이 크거나 cg가 화려한 영화들은 \n\xa0\n지금까지 후속작에서도 성공을 해왔다.\n\xa0\n마찬가지로\xa0아바타2도 성공할것이다.\n\xa0\n\xa0\nii ) 스파이더맨 4 (어메이징 스파이더맨?)\n\xa0\n정말 기다려지는 영화중 하나이다.\n\xa0\n현재 소식으로는 2012년 7월달에 개봉한다는데\n\xa0\n1년 조금 넘게 남았네.\n\xa0\n아..미래 사람들은\xa0이미 대부분 다 봤겠지만\n\xa0\n아직\xa0전 안봤습니다...이해 해주세요\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n예측 2.\xa0\xa0(정치)\n\xa0\n현재 이명박 씨가 대통령이다.\n\xa0\n내년(2012년)이면 대통령 선거를 한다.\n\xa0\n내 생각엔 박근혜가 대통령이 될

In [38]:
# 한글을 제외하고 전부 제거
test_sample = re.sub(r"[^가-힣]", "", test_sample)
test_sample

'년에아바타가개봉하다니아바타정말재밌게봤었는데물론지금이글을쓰고있는시간은년월이지만다시생각해도아바타정말걸작중의걸작이다편은판도라바다를배경으로만든다는데아무튼재밌겠다보러가야지현재년에는가유행인데년엔나오려나년뒤니깐나올수도있겠지지금으로부터년전년만해도영화를로본다는건상상도못했었는데암튼과학기술의발달은정말빠른것같다이제미래에대한몇가지예측들을해보겠다년예측영화아바타대부분편에서흥행성공한영화들은후속작에서망하던데아바타도과연그럴까내생각엔아바타도흥행성공할것같다왜냐하면스케일이크거나가화려한영화들은지금까지후속작에서도성공을해왔다마찬가지로아바타도성공할것이다스파이더맨어메이징스파이더맨정말기다려지는영화중하나이다현재소식으로는년월달에개봉한다는데년조금넘게남았네아미래사람들은이미대부분다봤겠지만아직전안봤습니다이해해주세요예측정치현재이명박씨가대통령이다내년년이면대통령선거를한다내생각엔박근혜가대통령이될것같다예측국제년후반기년안에몇차례북한의남한도발이있을것같다그로인해많은피해자가생길것이다최근북한의도발사건으로는연평도사건이나천안함사건이있음예측은여기까지일종의타임캡슐이니년전에열어보시면안됩니다만약제가그때다시이글을발견한다면댓글을달겠습니다지금까지년에살아있을지모르는년사람의글이었습니다'

In [39]:
# 세글자 이상의 명사만 가지고 오기
tokens = okt.nouns(test_sample)
cleaned_tokens = list()

for word in tokens:
    if len(word) >= 3:
        cleaned_tokens.append(word)
    else:
        pass
cleaned_review = ' '.join(cleaned_tokens)
        
cleaned_review   

'아바타 아바타 아바타 판도라 년전년 과학기술 아바타 대부분 후속작 아바타 아바타 스케일 후속작 마찬가지 아바타 스파이더맨 어메이징 스파이더맨 이명박 대통령 대통령선거 박근혜 대통령 후반기 피해자 연평도 천안함 타임캡슐'

In [40]:
# TF-IDF 변환기에 데이터를 입력하여 변환
test_review_tfid = tfid.transform([cleaned_review])

# 배열을 출력
print(test_review_tfid)

  (0, 593)	0.6060391236784104
  (0, 558)	0.2859920478400769
  (0, 260)	0.6060391236784104
  (0, 134)	0.3030195618392052
  (0, 91)	0.3030195618392052


In [41]:
# 로지스틱 회귀 모델을 활용하여 분류 예측
test_pred = lr.predict(test_review_tfid)[0]
print("분석 결과 {}적인 리뷰로 예측됩니다. ".format("긍정" if test_pred > 0 else "부정")) #-> 긍정으로보임 

분석 결과 긍정적인 리뷰로 예측됩니다. 
